In [1]:
%matplotlib inline

In [2]:
from importlib import reload

# 4. Force layout

There exists different python package for force layout based techniques.

The two I tried were: 
- https://github.com/Iain530/force-directed-layout-algorithms
- https://github.com/bhargavchippada/forceatlas2

# Force layout

The expression for the stress to optimise is: $stress(X) = \sum_{i<j}w_{ij}(||X_i - X_j|| - d_{ij})^2$ with $w_{ij} = d_{ij}^{-2}$, $||X_i - X_j||$ the euclidean norm computed in the low-dimensional embedding and $d_{ij}$ representing the pairwise distance in the high-dimensional embedding.

The gradient for this cost has an analytical solution equal to:

$grad_i(stress(X_i)) = \sum_j 4 w_{ij} \frac{(X_i - X_j)}{||X_i - X_j||} (||X_i - X_j|| - d_{ij})$

This expression can be computed in different ways. An explicit way does implement the computation for the gradient of the stress:

```python
def force(self):
    # hd_ij, ld_ij: pairwise distances in high, low dimensional space resp.

    pdist = ss.distance.pdist(self, 'euclidean')
    ld_ij_2 = ss.distance.squareform(pdist)

    F_ij_2 = (ld_ij_2 - self.hd_ij) / (ld_ij_2 + EPSILON)
    F_ij = F_ij_2[:,:] * self.inv_hd_ij_2

    a = self[:, None, :]
    ld_ij_1 = (a - a.swapaxes(0,1))

    F_ij = F_ij[:,:,None] * ld_ij_1

    F_i = np.sum(F_ij, axis=1)

    return 4*F_i
```

It is also possible to use autograd to compute the gradient of a given cost:


```python
def stress(X):
    X = X[:, None, :]
    ld_ij_2 = np.sqrt(np.sum(((X - X.swapaxes(0,1))**2), axis=2) + EPSILON)
    res = np.sum((ld_ij_2 - hd_ij)**2 * inv_hd_ij_2)
    return res

gradient = grad(stress)
```

Similar to a N-body simulation, an optimisation can take place in which the gradient descent is used.

```python
def gd(grad, x, callback=None, num_iters=200):
    for i in range(num_iters):
        g = grad()
        x -= g/2  # if full gradient, optimisation is unstable
    return x

def sgd(grad, x, callback=None, num_iters=200, step_size=0.1, mass=0.9):
    velocity = np.zeros(x.shape)
    for i in range(num_iters):
        g = grad()
        if callback: callback(x, i, g)
        velocity = mass * velocity - (1.0 - mass) * g
        x += step_size * velocity
    return x
```